In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import os

class EmployeeEditor:
    def __init__(self, username, password, driver_path, headless=True):
        self.username = username
        self.password = password
        self.base_url = "https://employees.iedit.mpg.de"
        self.driver_path = driver_path

        # Set up Firefox options
        firefox_options = Options()
        if headless:
            firefox_options.add_argument("--headless")  # Ensure GUI is off
        firefox_options.add_argument("--no-sandbox")
        firefox_options.add_argument("--disable-dev-shm-usage")

        # Initialize the WebDriver
        print("Initializing WebDriver")
        self.driver = webdriver.Firefox(service=Service(self.driver_path), options=firefox_options)
        self.wait = WebDriverWait(self.driver, 10)

    def login(self):
        login_url = f"{self.base_url}/login?locale=de"
        print(f"Navigating to {login_url}")
        self.driver.get(login_url)

        # Find and fill in the email field
        email_field = self.wait.until(EC.presence_of_element_located((By.ID, "email")))
        email_field.clear()
        email_field.send_keys(self.username)

        # Find and fill in the password field
        password_field = self.driver.find_element(By.ID, "password")
        password_field.clear()
        password_field.send_keys(self.password)

        # Find and click the submit button
        submit_button = self.driver.find_element(By.XPATH, "//input[@type='submit' and @value='Anmelden']")
        submit_button.click()

        # Wait for login to complete
        self.wait.until(EC.presence_of_element_located((By.CLASS_NAME, "logout")))
        print("Login successful")

    def search_employee(self, search_query):
        employees_url = f"{self.base_url}/de/employees"
        print(f"Navigating to {employees_url}")
        self.driver.get(employees_url)

        # Find the search input field and enter the search query
        search_field = self.wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='search']")))
        search_field.clear()
        search_field.send_keys(search_query)
        search_field.send_keys(Keys.RETURN)

        # Wait for the table to be populated with results
        while True:
            try:
                import time
                time.sleep(4)
                # Wait until the table rows are loaded
                self.wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#employees_index_de tbody tr")))

                # Print the whole table
                table = self.driver.find_element(By.CSS_SELECTOR, "#employees_index_de")
                print("Table HTML content:")
                print(table.get_attribute('outerHTML'))

                # Find the element containing the search string (name)
                employee_element = self.driver.find_element(By.XPATH, f"//*[contains(text(), '{search_query}')]")

                # Ensure the element is not the search box itself
                if employee_element.tag_name == 'input' and employee_element.get_attribute('type') == 'search':
                    continue

                # Click on the employee element
                employee_element.click()
                print(f"Clicked on the element containing: {search_query}")
                break
            except NoSuchElementException:
                print("No matching entries found yet, retrying...")
                self.driver.refresh()  # Refresh the page and try again
                self.wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#employees_index_de tbody tr")))

    def run(self, search_query):
        self.login()
        self.search_employee(search_query)


# Example usage
username = os.getenv("USERNAME_EMPLOYEE_DB")
password = os.getenv("PASSWORD_EMPLOYEE_DB")
search_query = "Franz Roters"

# Path to your manually downloaded geckodriver
driver_path = "../geckodriver.exe"

# Set headless to False if you want to see the browser window
editor = EmployeeEditor(username, password, driver_path, headless=False)
editor.run(search_query)


Initializing WebDriver
Navigating to https://employees.iedit.mpg.de/login?locale=de
Login successful
Navigating to https://employees.iedit.mpg.de/de/employees


AttributeError: module 'time' has no attribute 'sleep4'

In [ ]:
# from pubman_manager import EmployeeEditor
# import os
# import pytest

# from dotenv import load_dotenv
# load_dotenv()
# username = os.getenv("USERNAME_EMPLOYEE_DB")
# password = os.getenv("PASSWORD_EMPLOYEE_DB")
# a = EmployeeEditor(username, password)
# a.login()
# a.search_employee('Franz Roters')


# # @pytest.fixture
# # def employee_api():
# #     from dotenv import load_dotenv
# #     load_dotenv()
# #     username = os.getenv("USERNAME_EMPLOYEE_DB")
# #     password = os.getenv("PASSWORD_EMPLOYEE_DB")
# #     return EmployeeDBAPI(username, password)

# # def test_process_excel_and_create_publications(employee_api):
# #     employee_api.login()

# # if __name__ == "__main__":
# #     pytest.main(["-v"])


TypeError: EmployeeEditor.__init__() takes 3 positional arguments but 4 were given